In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import ptb
import re

In [20]:
import re

text = "Python is a powerful language"
words = re.split(';|,| ', text)
print(words)

['Python', 'is', 'a', 'powerful', 'language']


In [31]:
corpus_raw = 'He is the king? The king is royal. She is the royal queen'
re.split('\.|\? | !', corpus_raw)

['He is the king', 'The king is royal', ' She is the royal queen']

In [35]:
def preprocess(text):
    text = text.lower() #to lowercase
    corpus = re.split('\.|\?|!', text) #split sentence
    word_to_id = {}
    id_to_word = {}
    for sentence in corpus:
        sentence = sentence.split()
        for word in sentence: #create word_to_id and id_to_word
            if word not in word_to_id:
                id = len(word_to_id)
                word_to_id[word] = id
                id_to_word[id] = word

    for i in range(len(corpus)): #create corpus
        corpus[i] = [word_to_id[word] for word in corpus[i].split()]

    return corpus, word_to_id, id_to_word
corpus, word_to_id, id_to_word = preprocess(corpus_raw)
print(corpus)
print(word_to_id)


[[0, 1, 2, 3], [2, 3, 1, 4], [5, 1, 2, 4, 6]]
{'he': 0, 'is': 1, 'the': 2, 'king': 3, 'royal': 4, 'she': 5, 'queen': 6}
